In [2]:
import pyvista as pv
import pyacvd
import matplotlib.pyplot as plt
import numpy as np
import trimesh
import math
from scipy.optimize import minimize
import os
import json
import meshpy.triangle as triangle
from scipy.spatial import distance
from numpy.linalg import svd, norm
from matplotlib.path import Path
from scipy.spatial import Delaunay
from scipy.interpolate import RBFInterpolator


In [3]:
wdir = '/Users/rubi/Documents/GitHub/Chloroplast'
#wdir = '/martini/rubiz/Chloroplast'


In [4]:
def load_obj_with_materials(obj_path, mtl_path=None):
    """Load OBJ file and return PyVista mesh with material colors as attributes.
    
    Parameters
    ----------
    obj_path : str
        Path to the OBJ file.
    mtl_path : str, optional
        Path to the MTL file containing material definitions.

    Returns
    -------
    pv_mesh : pyvista.PolyData or list of pyvista.PolyData
        PyVista mesh(es) with material colors as point/cell data attributes.
        Returns single mesh if OBJ contains one object, list if multiple.
    """
    # Parse materials if MTL file provided
    materials = {}
    if mtl_path:
        materials = parse_mtl_file(mtl_path)
    
    # Load mesh with trimesh
    scene = trimesh.load(obj_path)
    
    if isinstance(scene, trimesh.Scene):
        # Handle multiple objects in scene
        meshes = []
        
        for name, mesh in scene.geometry.items():
            # Create PyVista mesh manually to avoid UV issues
            vertices = np.asarray(mesh.vertices)
            faces = np.asarray(mesh.faces)
            cells = np.column_stack([np.full(faces.shape[0], 3), faces]).flatten()
            pv_mesh = pv.PolyData(vertices, cells)
            
            # Add material information as mesh attributes
            if hasattr(mesh, 'visual') and hasattr(mesh.visual, 'material'):
                if hasattr(mesh.visual.material, 'name'):
                    material_name = mesh.visual.material.name
                    pv_mesh['material_name'] = [material_name] * pv_mesh.n_points
                    
                    # Add color information if material exists
                    if material_name in materials:
                        diffuse_color = materials[material_name]['diffuse']
                        # Add as point data (RGB values repeated for each point)
                        pv_mesh['material_color'] = np.tile(diffuse_color, (pv_mesh.n_points, 1))
            
            # Add face colors if available from trimesh
            if hasattr(mesh, 'visual') and hasattr(mesh.visual, 'face_colors'):
                if mesh.visual.face_colors is not None:
                    face_colors = np.asarray(mesh.visual.face_colors)
                    if face_colors.max() > 1.0:
                        face_colors = face_colors / 255.0  # Normalize to 0-1
                    pv_mesh['face_colors'] = face_colors
            
            meshes.append(pv_mesh)
        
        return meshes if len(meshes) > 1 else meshes[0]
    
    else:
        # Single mesh
        vertices = np.asarray(scene.vertices)
        faces = np.asarray(scene.faces)
        cells = np.column_stack([np.full(faces.shape[0], 3), faces]).flatten()
        pv_mesh = pv.PolyData(vertices, cells)
        
        # Add material information if available
        if hasattr(scene, 'visual') and hasattr(scene.visual, 'material'):
            if hasattr(scene.visual.material, 'name'):
                material_name = scene.visual.material.name
                pv_mesh['material_name'] = [material_name] * pv_mesh.n_points
                
                if material_name in materials:
                    diffuse_color = materials[material_name]['diffuse']
                    pv_mesh['material_color'] = np.tile(diffuse_color, (pv_mesh.n_points, 1))
        
        # Add face colors if available
        if hasattr(scene, 'visual') and hasattr(scene.visual, 'face_colors'):
            if scene.visual.face_colors is not None:
                face_colors = np.asarray(scene.visual.face_colors)
                if face_colors.max() > 1.0:
                    face_colors = face_colors / 255.0
                pv_mesh['face_colors'] = face_colors
        
        return pv_mesh

def parse_mtl_file(mtl_path):
    """Parse MTL file to extract material colors.
    
    Parameters
    ----------
    mtl_path : str
        Path to the MTL file.

    Returns
    -------
    materials : dict
        Dictionary mapping material names to their diffuse RGB colors.
    """
    materials = {}
    current_material = None
    
    with open(mtl_path, 'r') as f:
        for line in f:
            line = line.strip()
            if line.startswith('newmtl'):
                current_material = line.split()[1]
                materials[current_material] = {}
            elif line.startswith('Kd') and current_material:
                # Diffuse color (RGB values 0-1)
                rgb = [float(x) for x in line.split()[1:4]]
                materials[current_material]['diffuse'] = rgb
            elif line.startswith('Ka') and current_material:
                # Ambient color (RGB values 0-1)
                rgb = [float(x) for x in line.split()[1:4]]
                materials[current_material]['ambient'] = rgb
            elif line.startswith('Ks') and current_material:
                # Specular color (RGB values 0-1)
                rgb = [float(x) for x in line.split()[1:4]]
                materials[current_material]['specular'] = rgb
    
    return materials

def create_periodic_copies(mesh, n_copies_x_positive=1, n_copies_x_negative=1,
                           n_copies_y_positive=1, n_copies_y_negative=1,
                           n_copies_z_positive=1, n_copies_z_negative=1, 
                           x_limits=[], y_limits=[], z_limits=[]):
    """Create periodic copies of a mesh in the specified directions.
    
    Parameters
    ----------
    mesh : pyvista.PolyData or list of pyvista.PolyData
        The mesh(es) to copy.
    n_copies_x_positive : int, default: 1
        Number of copies in the positive x direction.
    n_copies_x_negative : int, default: 1
        Number of copies in the negative x direction.
    n_copies_y_positive : int, default: 1
        Number of copies in the positive y direction.
    n_copies_y_negative : int, default: 1
        Number of copies in the negative y direction.
    n_copies_z_positive : int, default: 1
        Number of copies in the positive z direction.
    n_copies_z_negative : int, default: 1
        Number of copies in the negative z direction.
    x_limits : list, default: []
        List of two floats specifying the x limits [xmin, xmax]. If empty, uses mesh bounds.
    y_limits : list, default: []
        List of two floats specifying the y limits [ymin, ymax]. If empty, uses mesh bounds.
    z_limits : list, default: []
        List of two floats specifying the z limits [zmin, zmax]. If empty, uses mesh bounds.
        
    Returns
    -------
    list of pyvista.PolyData
        List containing the original mesh and all periodic copies.
    """
    # Handle both single mesh and list of meshes
    if isinstance(mesh, list):
        mesh_list = mesh
    else:
        mesh_list = [mesh]
    
    # Calculate dimensions for translation
    if x_limits:
        x_min, x_max = x_limits
        x_size = x_max - x_min
    else:
        # Get bounds from all meshes
        all_bounds = []
        for m in mesh_list:
            all_bounds.append(m.bounds)
        all_bounds = np.array(all_bounds)
        x_min = all_bounds[:, 0].min()  # minimum x
        x_max = all_bounds[:, 1].max()  # maximum x
        x_size = x_max - x_min
    
    if y_limits:
        y_min, y_max = y_limits
        y_size = y_max - y_min
    else:
        y_min = all_bounds[:, 2].min()  # minimum y
        y_max = all_bounds[:, 3].max()  # maximum y
        y_size = y_max - y_min
    
    if z_limits:
        z_min, z_max = z_limits
        z_size = z_max - z_min
    else:
        z_min = all_bounds[:, 4].min()  # minimum z
        z_max = all_bounds[:, 5].max()  # maximum z
        z_size = z_max - z_min
    
    all_meshes = []
    
    # Generate all translation vectors
    for i in range(-n_copies_x_negative, n_copies_x_positive + 1):
        for j in range(-n_copies_y_negative, n_copies_y_positive + 1):
            for k in range(-n_copies_z_negative, n_copies_z_positive + 1):
                
                # Calculate translation vector
                translation = np.array([
                    i * x_size,
                    j * y_size,
                    k * z_size
                ])
                
                # Create copies of all meshes with this translation
                for original_mesh in mesh_list:
                    if i == 0 and j == 0 and k == 0:
                        # Original mesh (no translation)
                        all_meshes.append(original_mesh.copy())
                    else:
                        # Create translated copy using manual translation
                        translated_mesh = original_mesh.copy()
                        translated_mesh.points = translated_mesh.points + translation
                        all_meshes.append(translated_mesh)
    
    return all_meshes  

def create_periodic_copies(mesh, n_copies_x_positive=1, n_copies_x_negative=1,
                           n_copies_y_positive=1, n_copies_y_negative=1,
                           n_copies_z_positive=1, n_copies_z_negative=1, 
                           x_limits=[], y_limits=[], z_limits=[]):
    """Create periodic copies of a mesh in the specified directions.
    
    Parameters
    ----------
    mesh : pyvista.PolyData or list of pyvista.PolyData
        The mesh(es) to copy.
    n_copies_x_positive : int, default: 1
        Number of copies in the positive x direction.
    n_copies_x_negative : int, default: 1
        Number of copies in the negative x direction.
    n_copies_y_positive : int, default: 1
        Number of copies in the positive y direction.
    n_copies_y_negative : int, default: 1
        Number of copies in the negative y direction.
    n_copies_z_positive : int, default: 1
        Number of copies in the positive z direction.
    n_copies_z_negative : int, default: 1
        Number of copies in the negative z direction.
    x_limits : list, default: []
        List of two floats specifying the x limits [xmin, xmax]. If empty, uses mesh bounds.
    y_limits : list, default: []
        List of two floats specifying the y limits [ymin, ymax]. If empty, uses mesh bounds.
    z_limits : list, default: []
        List of two floats specifying the z limits [zmin, zmax]. If empty, uses mesh bounds.
        
    Returns
    -------
    list of pyvista.PolyData
        List containing the original mesh and all periodic copies.
    """
    # Handle both single mesh and list of meshes
    if isinstance(mesh, list):
        mesh_list = mesh
    else:
        mesh_list = [mesh]
    
    # Calculate dimensions for translation
    if x_limits:
        x_min, x_max = x_limits
        x_size = x_max - x_min
    else:
        # Get bounds from all meshes
        all_bounds = []
        for m in mesh_list:
            all_bounds.append(m.bounds)
        all_bounds = np.array(all_bounds)
        x_min = all_bounds[:, 0].min()  # minimum x
        x_max = all_bounds[:, 1].max()  # maximum x
        x_size = x_max - x_min
    
    if y_limits:
        y_min, y_max = y_limits
        y_size = y_max - y_min
    else:
        y_min = all_bounds[:, 2].min()  # minimum y
        y_max = all_bounds[:, 3].max()  # maximum y
        y_size = y_max - y_min
    
    if z_limits:
        z_min, z_max = z_limits
        z_size = z_max - z_min
    else:
        z_min = all_bounds[:, 4].min()  # minimum z
        z_max = all_bounds[:, 5].max()  # maximum z
        z_size = z_max - z_min
    
    all_meshes = []
    
    # Generate all translation vectors
    for i in range(-n_copies_x_negative, n_copies_x_positive + 1):
        for j in range(-n_copies_y_negative, n_copies_y_positive + 1):
            for k in range(-n_copies_z_negative, n_copies_z_positive + 1):
                
                # Calculate translation vector
                translation = np.array([
                    i * x_size,
                    j * y_size,
                    k * z_size
                ])
                
                # Create copies of all meshes with this translation
                for original_mesh in mesh_list:
                    if i == 0 and j == 0 and k == 0:
                        # Original mesh (no translation)
                        all_meshes.append(original_mesh.copy())
                    else:
                        # Create translated copy using manual translation
                        translated_mesh = original_mesh.copy()
                        translated_mesh.points = translated_mesh.points + translation
                        all_meshes.append(translated_mesh)
    
    return all_meshes  

def extract_model(model,zi,zf):
    """Extract a subgrid of the model based on z-coordinates.
    
    Parameters
    ----------
    model : pyvista.PolyData
        The input model to extract from.
    zi : float
        The lower z-coordinate limit.
    zf : float
        The upper z-coordinate limit.
    
    Returns
    -------
    pyvista.PolyData
        The extracted subgrid containing cells within the specified z limits.
    """
    # Get cell centers
    cell_centers = model.cell_centers().points
    # Create a mask for cells within the z limits
    mask = (cell_centers[:, 2] > zi) & (cell_centers[:, 2] < zf)
    # Get indices of cells that match the mask
    cell_ind = mask.nonzero()[0]
    # Extract the subgrid using the cell indices
    subgrid = model.extract_cells(cell_ind)
    return subgrid

def print_mesh_size(mesh):
    """Print the size of the mesh in each dimension.
    
    Parameters
    ----------
    mesh : pyvista.PolyData
        The mesh to analyze.
    """
    bounds = mesh.bounds
    dx = bounds[1] - bounds[0]
    dy = bounds[3] - bounds[2]
    dz = bounds[5] - bounds[4]
    print(f"Size of mesh: dx={dx}, dy={dy}, dz={dz}")


In [5]:
# Load mesh (.obj) and materials (.mtl)
obj_path = f"{wdir}/models/final_1nm2.obj"
mtl_path = f"{wdir}/models/final_1nm2.mtl"

# Load the meshes with materials
model = load_obj_with_materials(obj_path, mtl_path) 

merged_model = model[0]
for m in model[1:]:
    merged_model += m

merged_model = merged_model.rotate_z(45)


In [6]:
plot = pv.Plotter(window_size=[1800, 600])
plot.background_color = 'black'

colors = merged_model['material_color']
plot.add_mesh(merged_model, scalars=colors, rgb=True, show_edges=False)
plot.camera_position = 'xz'
plot.camera.parallel_projection = True
plot.show()

Widget(value='<iframe src="http://localhost:52802/index.html?ui=P_0x1443fafe0_0&reconnect=auto" class="pyvista…

In [10]:


zi= -25.0
zf = 50.0
dz = 75.0

periodic_model = extract_model(merged_model, zi, zf)
print_mesh_size(periodic_model)

Size of mesh: dx=436.4970386538728, dy=436.49703936097956, dz=77.262495


In [12]:
plot = pv.Plotter(window_size=[1800, 600])
plot.background_color = 'white'

colors = periodic_model['material_color']
plot.add_mesh(periodic_model, scalars=colors, rgb=True, show_edges=False)
plot.camera_position = 'xz'
plot.camera.parallel_projection = True
plot.show()

Widget(value='<iframe src="http://localhost:52802/index.html?ui=P_0x14fe4e980_3&reconnect=auto" class="pyvista…

In [ ]:
break 

In [ ]:

periodic = create_periodic_copies(
    [periodic_model],
    n_copies_x_positive=12,
    n_copies_x_negative=12,  
    n_copies_y_positive=0,
    n_copies_y_negative=0,
    n_copies_z_positive=6,
    n_copies_z_negative=6,
    z_limits=[zi, zf]
)

merged_periodic = periodic[0]
for m in periodic[1:]:
    merged_periodic += m

# Print size
bounds = merged_periodic.bounds
dx = bounds[1] - bounds[0]
dy = bounds[3] - bounds[2]
dz = bounds[5] - bounds[4]
print(f"Size of periodic mesh: dx={dx}, dy={dy}, dz={dz}")



In [ ]:
plot2 = pv.Plotter(window_size=[1800, 600])
plot2.background_color = 'white'
# Add periodic copies
colors = merged_periodic['material_color']
plot2.add_mesh(merged_periodic, scalars=colors, rgb=True, opacity=1, show_scalar_bar=False)
plot2.camera_position = 'xz'
plot2.camera.parallel_projection = True
plot2.show()

Widget(value='<iframe src="http://localhost:51028/index.html?ui=P_0x3a87a0e50_35&reconnect=auto" class="pyvist…

In [ ]:
plot2 = pv.Plotter(window_size=[1800, 600])
plot2.background_color = 'black'
# Add periodic copies
for copy in periodic:
    periodic_grana_plot.add_mesh(periodic, color='yellow', opacity=1)
plot2.camera_position = 'xz'
plot2.camera.parallel_projection = True
plot2.show()

NotImplementedError: Unable to wrap (<class 'list'>) into a pyvista type.